In [16]:
import numpy as np
import pandas as pd

<b> Загрузка и предобработка исходных данных

In [37]:
data = pd.read_csv("../../Практические материалы/Lab 1. Linear and logistic regression/mnist/mnist_train.csv")
data_test = pd.read_csv("../../Практические материалы/Lab 1. Linear and logistic regression/mnist/mnist_test.csv")

In [38]:
data.head()

,label,1x1,1x2,1x3,1x4,1x5,1x6,1x7,1x8,1x9,...,28x19,28x20,28x21,28x22,28x23,28x24,28x25,28x26,28x27,28x28
0,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [39]:
X = data.iloc[:,1:].values
X_test = data_test.iloc[:,1:].values

In [40]:
mean = np.mean(X)
std = np.std(X)

X = (X - mean)/std
X_test = (X_test - mean)/std

In [41]:
m = len(X)
n = 10

Y = np.zeros([m,n])

for i in range(m):
    Y[i,data.label.values[i]] = 1

In [42]:
Y

array([[0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.]])

In [186]:
def sigmoid(z):
    return 1/(1+np.exp(-z))

In [187]:
def tanh(z):
    return (np.exp(z)-np.exp(-z))/(np.exp(z)+np.exp(-z))

In [188]:
def relu(z):
    return np.max(0,z)

In [189]:
def softmax(z):
    return np.exp(z)/np.sum(np.exp(z), axis = 1).reshape([len(z),1])

In [190]:
def conv(z, filters, padding = 1):
    height = z.shape[0]
    width = z.shape[1]
    depth = z.shape[2]
    samples = z.shape[3]
    n = filters.shape[0]
    m = filters.shape[1]
    l = filters.shape[2]
    n_filters = filters.shape[3]
    
    z_pad = np.zeros([height+padding*2,width+padding*2,depth,samples])
    if padding > 0:
        z_pad[padding:-padding,padding:-padding,:,:] = z[:,:,:,:]
    else:
        z_pad = z
    
    output_h = height + padding * 2 - (n - 1)
    output_w = width + padding * 2 - (n - 1)
    result = np.zeros([output_h,output_w,n_filters,samples])
    
    for s in range(samples):
        for k in range(n_filters):
            for i in range(output_h):
                for j in range(output_w):
                    result[i,j,k,s] = np.sum(z_pad[i:i+n,j:j+m,0:l,s]*filters[:,:,:,k])

    return result
    

In [191]:
def max_pool(z, pool_size):
    height = z.shape[0]
    width = z.shape[1]
    depth = z.shape[2]
    samples = z.shape[3]
    
    h2 = (pool_size - height%pool_size)%pool_size
    w2 = (pool_size - width%pool_size)%pool_size
        
    z_pad = np.zeros([height+h2,width+w2,depth,samples])
    z_pad[:height,:width,:,:] = z[:,:,:,:]

    result = np.zeros([(height+h2)//pool_size,(width+w2)//pool_size,depth,samples])
        
    for s in range(samples):    
        for k in range(depth):
            for i in range(0,height+h2,pool_size):
                for j in range(0,width+w2,pool_size):
                    tmp = z_pad[i:i+pool_size,j:j+pool_size,k,s]
                    result[i//pool_size,j//pool_size,k,s] = np.max(tmp)
                    #print('tmp',np.max(tmp))
                    #print(tmp.shape)
                    max_ind = np.argmax(tmp.flatten())
                    #print(max_ind//pool_size,max_ind%pool_size)
                    
    return result

In [192]:
def avg_pool(z, pool_size):
    height = z.shape[0]
    width = z.shape[1]
    depth = z.shape[2]
    samples = z.shape[3]
    
    h2 = (pool_size - height%pool_size)%pool_size
    w2 = (pool_size - width%pool_size)%pool_size
        
    z_pad = np.zeros([height+h2,width+w2,depth,samples])
    z_pad[:height,:width,:,:] = z[:,:,:,:]

    result = np.zeros([(height+h2)//pool_size,(width+w2)//pool_size,depth,samples])
    
    argmax_ind = np.zeros([(height+h2)//pool_size,(width+w2)//pool_size,depth,samples,2])

    for s in range(samples):
        for k in range(depth):
            for i in range(0,height+h2,pool_size):
                for j in range(0,width+w2,pool_size):
                    tmp = z_pad[i:i+pool_size,j:j+pool_size,k,s]
                    result[i//pool_size,j//pool_size,k,s] = np.mean(tmp)

                    max_ind = np.argmax(tmp.flatten())

                    argmax_ind[i//pool_size,j//pool_size,k,s,:] = [max_ind//pool_size,max_ind%pool_size]

    return result

In [324]:
def flatten(z):
    dim0 = z.shape[0]
    dim1 = z.shape[1]
    dim2 = z.shape[2]
    dim3 = z.shape[3]
    
    result = np.zeros([dim3,dim0*dim1*dim2])
    
    for s in range(dim3):
        result[s,:] = z[:,:,:,s].flatten()
    
    return result

In [325]:
def d_conv(filters,f_prev,sigma,padding = 1):
    d_filters = np.zeros_like(filters)
    height = sigma.shape[0]
    width = sigma.shape[1]
    depth = sigma.shape[2]
    samples = sigma.shape[3]
    n = filters.shape[0]
    m = filters.shape[1]
    l = filters.shape[2]
    n_filters = filters.shape[3]
    
    f_prev_pad = np.zeros([f_prev.shape[0]+padding*2,f_prev.shape[1]+padding*2,f_prev.shape[2],f_prev.shape[3]])
    
    if padding > 0:
        f_prev_pad[padding:-padding,padding:-padding,:,:] = f_prev[:,:,:,:]
    else:
        f_prev_pad = f_prev
    
    #print(X)
    sigma_new = np.zeros_like(f_prev_pad)
    
    for s in range(samples):
        for k in range(n_filters):
            for i in range(height):
                for j in range(width):
                    d_filters[:,:,:,k] += f_prev_pad[i:i+n,j:j+m,0:l,s]*sigma[i,j,k,s]
                    sigma_new[i:i+n,j:j+m,0:l,s] += filters[:,:,:,k]*sigma[i,j,k,s]
                
    if padding > 0:
        sigma_new = sigma_new[padding:-padding,padding:-padding,:,:]
                
    return d_filters,sigma_new

In [348]:
def d_flat(z,size):
    dim0 = size[0]
    dim1 = size[1]
    dim2 = size[2]
    dim3 = size[3]
    
    result = np.zeros(size)
    
    for s in range(dim3):
        result[:,:,:,s] = z[s,:].reshape([dim0,dim1,dim2])
    
    return result

In [349]:
def d_max_pool(sigma,f_prev,pool_size):
    result = np.zeros([sigma.shape[0]*pool_size,sigma.shape[1]*pool_size,sigma.shape[2],sigma.shape[3]])

    for s in range(sigma.shape[3]):
        for k in range(sigma.shape[2]):
            for i in range(0,sigma.shape[0]*pool_size,pool_size):
                for j in range(0,sigma.shape[1]*pool_size,pool_size):
                    tmp = f_prev[i:i+pool_size,j:j+pool_size,k,s]
                    max_ind = np.argmax(tmp.flatten())
                    
                    result[i+max_ind//pool_size,j+max_ind%pool_size,k,s] = sigma[i//pool_size,j//pool_size,k,s]

    return result

In [350]:
def d_avg_pool(sigma,f_prev,pool_size):
    result = np.zeros([sigma.shape[0]*pool_size,sigma.shape[1]*pool_size,sigma.shape[2],sigma.shape[3]])

    for s in range(sigma.shape[3]):
        for k in range(sigma.shape[2]):
            for i in range(0,sigma.shape[0]*pool_size,pool_size):
                for j in range(0,sigma.shape[1]*pool_size,pool_size):
                    result[i:i+pool_size,j:j+pool_size,k,s] = sigma[i//pool_size,j//pool_size,k,s]

    return result/pool_size**2

In [351]:
def forward(filters,X,W,B,pool_size):
    F0 = conv(X, filters[0], padding = 2)
    F1 = tanh(F0)
    F2 = avg_pool(F1,pool_size)
    F3 = conv(F2, filters[1], padding = 0)
    F4 = tanh(F3)
    F5 = avg_pool(F4,pool_size)
    F6 = conv(F5, filters[2], padding = 0)
    F7 = tanh(F6)
    F8 = flatten(F7)
    F9 = np.matmul(F8,W[0]) + B[0]
    F10 = sigmoid(F9)
    F11 = np.matmul(F10,W[1]) + B[1]
    F12 = sigmoid(F11)
    
    return (F0,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,F11,F12)

In [372]:
def backprop(X,F,W,Y,filters,pool_size):
    sigma = (F[12] - Y)/(F[12]*(1-F[12]))
    sigma = sigma*F[12]*(1-F[12])
    
    #print(sigma.shape)
    
    dW2 = np.matmul(np.transpose(F[10]),sigma)
    dB2 = np.sum(sigma, axis = 0)
    
    sigma = np.matmul(sigma,np.transpose(W[1]))
    sigma = sigma*F[10]*(1-F[10])
    
    #print(sigma.shape)
    
    dW1 = np.matmul(np.transpose(F[8]),sigma)
    dB1 = np.sum(sigma, axis = 0)
    
    #print(F[8].shape,sigma.shape)
    
    sigma = np.matmul(sigma,np.transpose(W[0]))
    sigma = d_flat(sigma,F[7].shape)
    
    #print(sigma.shape)
    
    sigma = sigma*(1-F[7]**2)
    
    d_filters3,sigma = d_conv(filters[2],F[5],sigma,padding = 0)
    sigma = d_avg_pool(sigma,F[5],pool_size)
    
    sigma = sigma*(1-F[4]**2)
    
    d_filters2,sigma = d_conv(filters[1],F[2],sigma,padding = 0)
    sigma = d_avg_pool(sigma,F[2],pool_size)
    
    sigma = sigma*(1-F[1]**2)
    
    d_filters1,sigma = d_conv(filters[0],X,sigma,padding = 2)

    #d_filters = d_conv(filters,X,sigma,padding = 0)
    
    return (dW2,dB2,dW1,dB1,d_filters3,d_filters2,d_filters1)
    

In [373]:
np.random.seed(34)

X = np.random.randn(28,28,1,5)
filters1 = np.random.randn(5,5,1,6)
filters2 = np.random.randn(5,5,6,16)
filters3 = np.random.randn(5,5,16,120)

filters = [filters1,filters2,filters3]

pool_size = 2

W1 = np.random.randn(120,84)
B1 = np.zeros([1,84])

W2 = np.random.randn(84,10)
B2 = np.zeros([1,10])

W = [W1,W2]
B = [B1,B2]

F = forward(filters,X,W,B,pool_size)

Y = np.zeros_like(F[-1])

grad = backprop(X,F,W,Y,filters,pool_size)

J = -np.sum(Y*np.log(F[-1])+(1-Y)*np.log(1-F[-1]))
#J = 0.5*np.sum((Y-F[-1])**2)

for i in range(3):
    for j in range(3):
        F = forward(filters,X,W,B,pool_size)
        filters1[i,j,0,0] += 0.00000001
        #W2[i,j] += 0.00000001
        J1 = -np.sum(Y*np.log(F[-1])+(1-Y)*np.log(1-F[-1]))
        #J1 = 0.5*np.sum((Y-F[-1])**2)
        F = forward(filters,X,W,B,pool_size)
        J2 = -np.sum(Y*np.log(F[-1])+(1-Y)*np.log(1-F[-1]))
        #J2 = 0.5*np.sum((Y-F[-1])**2)
        filters1[i,j,0,0] -= 0.00000001
        #W2[i,j] -= 0.00000001
        #print("**************************************")
        print((J2-J1)/0.00000001,'\t',grad[6][i,j,0,0])
        #print("**************************************")

-28.040477673130226 	 -28.055661910388192
8.660623507239507 	 8.64654393289801
14.315963881017524 	 14.314131585941935
-36.23730435720063 	 -36.23458774385862
2.7184853479411686 	 2.716024580974947
-6.677298358681583 	 -6.697221318121266
35.78509364388083 	 35.78131759418134
49.8100149570746 	 49.79845271780735
4.439293377345166 	 4.420358996564994
